# Post Minimal Preprocessing script for ClinicalActFlow study
Created October 2018, Luke Hearne.

The general pipeline includes:
- Downsampling of dense CIFTI files into a parcellation and conversion to .tsv
- Nuisance regression with multiple pipelines via Taku's code (amended to suit BIDS data directories & ICA-AROMA)
- GLM to estimate task betas (potentially fit this at the same time as the NRP)
- Calculation of resting-state FC

In [1]:
import os,sys
import numpy as np
import pandas as pd
import postFmriprepPipelines as pfp
from importlib import reload # for debugging and rerunning with different parcs
pfp = reload(pfp) #for debugging

#global variables
PROJECT_DIR = '/projects/f_mc1689_1/ClinicalActFlow/'
TASKS = ['rest','bart','scap','stopsignal','taskswitch']
#PARC = 'cabn'
OVERWRITE = 0
#subject list
df = pd.read_csv('subject_list.txt', sep='\t', index_col = 0, header = 0)
SUBJ_LIST = df.index

# Parcellation

In [ ]:
def parcellateCIFTI(subj,parcellation_file,label,cifti_dir):
    for task in TASKS:
        subj_dir = cifti_dir+subj+'/MNINonLinear/Results/'+'task-'+task+'_bold/'
        input_file  = subj_dir+'task-'+task+'_bold_Atlas_s0.dtseries.nii'
        output_file = subj_dir+'task-'+task+'_bold_Atlas_s0_'+label+'.ptseries.nii'

        # check if it already exists
        if os.path.isfile(output_file + '.tsv') and OVERWRITE == 0:
            print('\t ...skipping subject ',subj,'|task: ',task)

        else:
            #downsample
            print('Downsampling grayordinate data to parcels for subject ',subj,'|task: ',task)
            cmd = 'wb_command -cifti-parcellate '+input_file+' '+parcellation_file+' COLUMN '+output_file+' -method MEAN'  
            os.system(cmd)

            #convert to .tsv
            input_file = output_file
            output_file = input_file + '.tsv'
            cmd = 'wb_command -cifti-convert -to-text ' + input_file + ' ' + output_file
            os.system(cmd)

if PARC is 'cabn':
    parcellation_file = '/projects/f_mc1689_1/AnalysisTools/ColeAnticevicNetPartition/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR.dlabel.nii'
    cifti_dir = PROJECT_DIR+'/data/prepro/ciftify/'
    for subj in SUBJ_LIST:
        parcellateCIFTI(subj,parcellation_file,PARC,cifti_dir)

# Nuisance and task GLMs

The *postFmriPrepPipelines* module contains code to perform nuisance and task regression on the data. 

See *postFmriPrepPipelines.py* for more information - the key variables are located in the script.

In [ ]:
# create nuisance regressors (this only needs to be done once)
for task in TASKS:
    pfp.step1_createNuisanceRegressors(task,nproc=8)

In [ ]:
# regress nuisance signal for fc estimation
pfp.step2_nuisanceRegression('rest', model='24pXaCompCorXVolterra',spikeReg=True,framesToSkip=4)

In [ ]:
# do task and nuis regression in a single step for both canonical and FIR models
for task in ['scap']:
    pfp.step2_taskRegression(task,task_model='canonical', model='24pXaCompCorXVolterra',spikeReg=False)
    #pfp.step2_taskRegression(task,task_model='FIR', model='24pXaCompCorXVolterra',spikeReg=False)

#pfp.step2_taskRegression('scap',task_model='canonical', model='24pXaCompCorXVolterra',spikeReg=False)

Running TASK: canonical regression on subject sub-10159 | task scap
	Model: 24pXaCompCorXVolterra with spikeReg: False | zscore:= False | Aggressive AROMA:= False
		 291  TRs (in resid)
		 718  rois (in resid)
Running TASK: canonical regression on subject sub-10171 | task scap
	Model: 24pXaCompCorXVolterra with spikeReg: False | zscore:= False | Aggressive AROMA:= False
		 291  TRs (in resid)
		 718  rois (in resid)
Running TASK: canonical regression on subject sub-10189 | task scap
	Model: 24pXaCompCorXVolterra with spikeReg: False | zscore:= False | Aggressive AROMA:= False
		 291  TRs (in resid)
		 718  rois (in resid)
Running TASK: canonical regression on subject sub-10217 | task scap
	Model: 24pXaCompCorXVolterra with spikeReg: False | zscore:= False | Aggressive AROMA:= False
		 291  TRs (in resid)
		 718  rois (in resid)
Running TASK: canonical regression on subject sub-10225 | task scap
	Model: 24pXaCompCorXVolterra with spikeReg: False | zscore:= False | Aggressive AROMA:= Fal

## Slurm for non-circular activation calculations

In [ ]:
# calculate non-circular activity based on vertices
from slurmUtils.py_to_slurm import *

file_name = 'noncircAct'
py_file = 'activity_noncircular_slurm.py' # located in slurmUtils
for i in range(len(SUBJ_LIST)):
    py_to_slurm_singlesub(SUBJ_LIST[i],file_name,py_file,time = '00:45:00',suppress_output=False)

# Calculate single-task FC

In [ ]:
pfp.step3_calcFC('rest',fc_method='partial_corr',model='24pXaCompCorXVolterra-spikeReg')
#pfp.step3_calcFC('rest',fc_method='pc_multregconn',model='24pXaCompCorXVolterra-spikeReg')

#pfp.step3_calcFC('bart',fc_method='pearsoncorr',model='24pXaCompCorXVolterra')
#pfp.step3_calcFC('taskswitch',fc_method='pearsoncorr',model='24pXaCompCorXVolterra')
#pfp.step3_calcFC('stopsignal',fc_method='pearsoncorr',model='24pXaCompCorXVolterra')
#pfp.step3_calcFC('scap',fc_method='pearsoncorr',model='24pXaCompCorXVolterra')

### Slurm for regression based FC's
Other than correlation these can be quite slow

In [ ]:
# this calculates fc for each task
from slurmUtils.py_to_slurm import *
file_name = 'FC'
py_file = 'fc_regression_slurm.py' # located in slurmUtils
for i in range(len(SUBJ_LIST)):
    py_to_slurm_singlesub(SUBJ_LIST[i],file_name,py_file,time = '00:5:00',suppress_output=False)

### Slurm for non-circular FC calculations
See the script (py_file) for specifics

In [ ]:
# calculate non-circular FC based on vertices
from slurmUtils.py_to_slurm import *

file_name = 'noncircFC'
py_file = 'fc_noncircular_slurm.py' # located in slurmUtils
for i in range(len(SUBJ_LIST)):
    py_to_slurm_singlesub(SUBJ_LIST[i],file_name,py_file,time = '00:30:00',suppress_output=False)

# Calculate multitask FC

In [ ]:
#pfp.step3_calcFCmulti(fc_method='multreg',zscore=True)
#pfp.step3_calcFCmulti(fc_method='pc_multregconn')

### Slurm for multitask regression FC

In [ ]:
from slurmUtils.py_to_slurm import *
file_name = 'FC'
py_file = 'fc_multi_regression_slurm.py' # located in slurmUtils
for i in range(len(SUBJ_LIST)):
    py_to_slurm_singlesub(SUBJ_LIST[i],file_name,py_file,time = '00:5:00',suppress_output=False)
#py_to_slurm_singlesub(SUBJ_LIST[0],file_name,py_file,time = '00:5:00',suppress_output=False)